# Code for Binary Equlibrium Optimizer with Simulated Annealing (version 1.2.1)
## Last updated: 16/07/2020
### Coders: Ritam Guha, Kushal Kanti Ghosh


### Import all the necessary libraries

In [1]:
import numpy as np
import pandas as pd
import time, sys

from sklearn.model_selection import train_test_split

from _utilities import compute_accuracy, initialize, save_excel, avg_concentration, sign_func, reshape_np, find_neighbor, onecount
from _utilities import Ufunction as transfer_func

from datetime import datetime

## Define the fitness function
There are two types - 
* fitness_FS: for feature selection
* fitness_Knapsack: for 0-1 knapsack

In [2]:
def fitness_FS(particles):
    # fitness computation for feature selection
    if(particles.ndim == 1):
        particles = reshape_np(particles)
        
    [num_particles, dimension] = particles.shape
    values = np.zeros((1, num_particles))
    count = 0
    
    for particle in particles:  
        current_particle = reshape_np(particle)
        set_cnt=int(np.sum(current_particle))
        
        if(set_cnt == 0):
            val = np.float64(1.0)     
            
        else:            
            set_cnt=set_cnt/dimension
            err = 1- compute_accuracy(train_X, train_Y, test_X, test_Y, particle)            
            val = omega*err + (1-omega)*set_cnt
        values[0, count] = np.float64(val) 
        count += 1            
    return values


def fitness_Knapsack(particles):
     # fitness computation for knapsack 
    if(particles.ndim == 1):
        particles = reshape_np(particles)
        
    [num_particles, dimension] = particles.shape
    values = np.zeros((1, num_particles))
    count = 0
    
    for particle in particles:  
        current_particle = reshape_np(particle)
        set_cnt=int(np.sum(current_particle))
        
        if(set_cnt == 0):
            val = np.float64(1.0)     
            
        else:            
            pos = np.flatnonzero(particle)                
            val = (np.sum(weights[pos])<=max_weight) * np.sum(worth[pos])
        values[0, count] = -np.float64(val) 
        count += 1            
    return values

## Simulated Annealing

In [3]:
def SA(population, fitness):
    # simulated annealing
    [particle_count, dimension] = population.shape
    T0 = dimension

    for particle_no in range(particle_count):
        T = 2*dimension
        current_particle = reshape_np(population[particle_no].copy())  
        current_fitness = compute_fitness(current_particle) 
        best_particle = current_particle.copy()
        best_fitness = current_fitness.copy()        
        
        while T>T0:
            new_particle = find_neighbor(current_particle)
            new_fitness = np.float64(compute_fitness(new_particle))            
            
            if new_fitness < best_fitness:
                current_particle = new_particle.copy()
                current_fitness = new_fitness.copy()
                best_particle = current_particle.copy()
                best_fitness = current_fitness.copy()
    
            else:            
                prob = np.exp((best_fitness-current_fitness)/T)
                if(np.random.random() <= prob):
                    current_particle = new_particle.copy()
                current_fitness = new_fitness
                
            T = int(T*0.7)                
        
        population[particle_no, :] = best_particle.copy() 
        fitness[0, particle_no] = best_fitness.copy()                   
        

## DEO with SA

In [4]:
def DEO_SA(init_population, dimension, pool_size=4, max_iter=50, particle_count=100, allow_SA=True):
    # pool initialization
    population = init_population.copy()
    eq_pool = np.zeros((pool_size+1, dimension))
    eq_fit = np.zeros((1, pool_size))
    eq_fit[0, :] = 100
    conv_plot = []
        
    # iterations start
    for iter in range(max_iter):        
        pop_new = np.zeros((particle_count, dimension))        
        fitness_list = compute_fitness(population) 
        sorted_idx = np.argsort(fitness_list)
        population = population[sorted_idx, :].reshape((particle_count, dimension))
        fitness_list = fitness_list[0, sorted_idx]

        # replacements in the pool
        for i in range(particle_count):
            for j in range(pool_size):                 
                if fitness_list[0, i] <= eq_fit[0, j]:
                    eq_fit[0, j] = fitness_list[0, i].copy()
                    eq_pool[j, :] = population[i, :].copy()
                    break
        

        conv_plot.append(eq_fit[0, 0])
        best_particle = eq_pool[0,:]
                
        Cave = avg_concentration(eq_pool,pool_size,dimension)
        eq_pool[pool_size] = Cave.copy()

        t = (1 - (iter/max_iter))**(a2*iter/max_iter)
        
        for i in range(particle_count): 
            # randomly choose one candidate from the equillibrium pool
            inx = np.random.randint(0,pool_size)
            Ceq = np.array(eq_pool[inx])

            lambda_vec = np.zeros(np.shape(Ceq))
            r_vec = np.zeros(np.shape(Ceq))
            for j in range(dimension):
                lambda_vec[j], r_vec[j] = np.random.random(2)

            F_vec = np.zeros(np.shape(Ceq))
            for j in range(dimension):
                x = -1*lambda_vec[j]*t 
                x = np.exp(x) - 1
                x = a1 * sign_func(r_vec[j] - 0.5) * x

            r1, r2 = np.random.random(2)
    
            if r2 < GP:
                GCP = 0
            else:
                GCP = 0.5 * r1

            G0 = np.zeros(np.shape(Ceq))
            G = np.zeros(np.shape(Ceq))

            for j in range(dimension):
                G0[j] = GCP * (Ceq[j] - lambda_vec[j]*population[i][j])
                G[j] = G0[j]*F_vec[j]
            
            # use transfer function to map continuous->binary
            for j in range(dimension):
                temp = Ceq[j] + (population[i][j] - Ceq[j])*F_vec[j] + G[j]*(1 - F_vec[j])/lambda_vec[j]                
                temp = transfer_func(temp)                
                if temp>np.random.random():
                    pop_new[i][j] = 1 - population[i][j]
                else:
                    pop_new[i][j] = population[i][j]                
        
        # performing simulated annealing
        if(allow_SA):            
            SA(pop_new, fitness_list)
    
            
        population = pop_new.copy()        
        
    return best_particle, conv_plot

In [5]:
def DEO_driver(dataset):
    # initializing components
    conv_plot_all = {}
    conv_plot = {}
    best_pop = -1
    best_iter = -1
    best_accuracy = -1
    best_no_features = -1      

    whole_accuracy = compute_accuracy(train_X, train_Y, test_X, test_Y, np.ones(dimension))
    print("Total Acc: ",whole_accuracy * 100)     

    # dictionary to store the final results
    final_result ={}    

    for particle_count in particle_count_testing:                
        for max_iter in max_iter_testing:     
            
            print("Particle count:", particle_count)
            print("Max iter:", max_iter)            
            
            accuracy_DEO = 0
            accuracy_DEOSA = 0
            best_accuracy_DEO = 0
            best_accuracy_DEOSA = 0
            avg_accuracy_DEO = 0
            avg_accuracy_DEOSA = 0
            accuracy_DEO_runs = []
            accuracy_DEOSA_runs = []

            num_features_DEO = 0
            num_features_DEOSA = 0   
            best_num_features_DEO = 0
            best_num_features_DEOSA = 0
            avg_num_features_DEO = 0
            avg_num_features_DEOSA = 0
            num_features_DEO_runs = []
            num_features_DEOSA_runs = []

            DEO_key = dataset + "_DEO" + "_pop_" + str(particle_count) + "_iter_" + str(max_iter)
            DEOSA_key = dataset + "_DEOSA" + "_pop_" + str(particle_count) + "_iter_" + str(max_iter)      

            for run in range(max_runs):
                
                # print("Run ", run+1)
                
                population = initialize(particle_count, dimension) 
                # print("DEO starts...")

                best_particle_DEO, conv_DEO = DEO_SA(init_population=population.copy(), pool_size=pool_size, max_iter=max_iter, particle_count=particle_count, dimension=dimension, allow_SA=False)            
                
                # print("DEOSA starts...")
                best_particle_DEOSA, conv_DEOSA = DEO_SA(init_population=population.copy(), pool_size=pool_size, max_iter=max_iter, particle_count=particle_count, dimension=dimension, allow_SA=True)                                                    

                conv_plot_all["DEO_pop_" + str(particle_count) + "_iter_" + str(max_iter)] = conv_DEO
                conv_plot_all["DEOSA_pop_" + str(particle_count)+ "_iter_" + str(max_iter)] = conv_DEOSA                        

                accuracy_DEO = compute_accuracy(train_X, train_Y, test_X, test_Y, best_particle_DEO) * 100
                accuracy_DEOSA = compute_accuracy(train_X, train_Y, test_X, test_Y, best_particle_DEOSA) * 100

                num_features_DEO = onecount(best_particle_DEO)
                num_features_DEOSA = onecount(best_particle_DEOSA)

                accuracy_DEO_runs.append(accuracy_DEO)
                accuracy_DEOSA_runs.append(accuracy_DEOSA)
                
                if(accuracy_DEO > best_accuracy_DEO):
                    best_accuracy_DEO = accuracy_DEO.copy()
                    best_num_features_DEO = np.float64(num_features_DEO).copy()
                    
                if(accuracy_DEOSA > best_accuracy_DEOSA):
                    best_accuracy_DEOSA = accuracy_DEOSA.copy()
                    best_num_features_DEOSA = np.float64(num_features_DEOSA).copy()

                avg_accuracy_DEO += accuracy_DEO
                avg_num_features_DEO += np.float64(num_features_DEO).copy()

                avg_accuracy_DEOSA += accuracy_DEOSA
                avg_num_features_DEOSA += np.float64(num_features_DEOSA).copy()


            std_deviation_accuracy_DEO = np.std(accuracy_DEO_runs)
            std_deviation_accuracy_DEOSA = np.std(accuracy_DEOSA_runs)

            std_deviation_accuracy_DEO = np.std(accuracy_DEO_runs)
            std_deviation_accuracy_DEOSA = np.std(accuracy_DEOSA_runs)

            avg_accuracy_DEO /= max_runs
            avg_num_features_DEO /= max_runs

            avg_accuracy_DEOSA /= max_runs
            avg_num_features_DEOSA /= max_runs
           
            if(best_accuracy_DEOSA > best_accuracy):
                best_pop = particle_count
                best_iter = max_iter                

            final_result[DEO_key + "best_accuracy"] = best_accuracy_DEO
            final_result[DEOSA_key + "best_accuracy"] = best_accuracy_DEOSA
            final_result[DEO_key + "avg_accuracy"] = avg_accuracy_DEO
            final_result[DEOSA_key + "avg_accuracy"] = avg_accuracy_DEOSA

            final_result[DEO_key + "best_num_features"] = best_num_features_DEO
            final_result[DEOSA_key + "best_num_features"] = best_num_features_DEOSA
            final_result[DEO_key + "avg_num_features"] = avg_num_features_DEO
            final_result[DEOSA_key + "avg_num_features"] = avg_num_features_DEOSA

            print('\n=================================================')
            print('     Results for (Pop - {}, Iter - {})'.format(particle_count, max_iter))
            print('=================================================')
            print('DEO best:', best_accuracy_DEO)
            print('DEO average:', avg_accuracy_DEO)
            print('DEO std:', std_deviation_accuracy_DEO)
            print('')
            print('DEOSA best:', best_accuracy_DEOSA)     
            print('DEOSA average:', avg_accuracy_DEOSA)  
            print('DEOSA std:', std_deviation_accuracy_DEOSA)   
            print('=================================================\n')

    # print("Best combination:" + " Pop - " + str(best_pop) + " Iter - " + str(best_iter))
    conv_plot["DEO"] = conv_plot_all["DEO_pop_" + str(best_pop) + "_iter_" + str(max_iter)]
    conv_plot["DEOSA"] = conv_plot_all["DEOSA_pop_" + str(best_pop) + "_iter_" + str(max_iter)]
        
    return final_result, conv_plot    

### Set the global parameters and run the following cell to perform feature selection over UCI datasets

* This code will automatically create two excel files - one for parameter variation and another one for convergence. The results will be saved as parameter_variation and convergence in Results folder

* It will also plot convergence curves which will be saved in Convergence folder


In [6]:
# set the global parameters over here
datasets=["BreastCancer","BreastEW","CongressEW","Exactly","Exactly2","HeartEW","Ionosphere","KrVsKpEW","Lymphography","M-of-n","PenglungEW","Sonar","SpectEW","Tic-tac-toe","Vote","WaveformEW","Wine","Zoo"]
# datasets=["WaveformEW","Wine","Zoo"]
# datasets=["BreastEW"]
particle_count_testing = [50]
max_iter_testing = [30]
max_runs = 10
omega = 0.9           
a2 = 1
a1 = 2
GP = 0.5
pool_size = 4
compute_fitness = fitness_FS

# initializing the excel files
save_excel(particle_count_testing, max_iter_testing, init=1, save_type="parameter variation")
save_excel(particle_count_testing, max_iter_testing, init=1, save_type="convergence")

# start iterating over the datasets
for dataset in datasets:
    
    # reading dataset
    print('\n===============================================================================')
    print("Dataset:", dataset)
    df=pd.read_csv("Data/"+dataset+".csv")
    (a,b)=np.shape(df)
    data = df.values[:,0:b-1]
    label = df.values[:,b-1]
    dimension = np.shape(data)[1] #particle dimension   
    print("dimension:", dimension)

    # loading_dataset
    cross = 5
    test_size = (1/cross)
    train_X, test_X, train_Y, test_Y = train_test_split(data, label,stratify=label ,test_size=test_size)

    final_result, conv_plot = DEO_driver(dataset)
    # plot_graph(final_result, conv_plot, dataset)

    save_excel(particle_count_testing, max_iter_testing, conv_plot, final_result, dataset, save_type="parameter variation")
    save_excel(particle_count_testing, max_iter_testing, conv_plot, final_result, dataset, save_type="convergence")
    print('===============================================================================\n')


Dataset: BreastCancer
dimension: 10
Total Acc:  60.71428571428571

Dataset: BreastEW
dimension: 30
Total Acc:  91.22807017543859

Dataset: CongressEW
dimension: 16
Total Acc:  91.95402298850574

Dataset: Exactly
dimension: 13
Total Acc:  76.5

Dataset: Exactly2
dimension: 13
Total Acc:  69.0

Dataset: HeartEW
dimension: 13
Total Acc:  74.07407407407408

Dataset: Ionosphere
dimension: 34
Total Acc:  88.57142857142857

Dataset: KrVsKpEW
dimension: 36
Total Acc:  96.08763693270735

Dataset: Lymphography
dimension: 18
Total Acc:  73.33333333333333

Dataset: M-of-n
dimension: 13
Total Acc:  88.5

Dataset: PenglungEW
dimension: 325
Total Acc:  93.33333333333333

Dataset: Sonar
dimension: 60
Total Acc:  88.09523809523809

Dataset: SpectEW
dimension: 22
Total Acc:  75.92592592592592

Dataset: Tic-tac-toe
dimension: 9
Total Acc:  84.89583333333334

Dataset: Vote
dimension: 16
Total Acc:  90.0

Dataset: WaveformEW
dimension: 40
Total Acc:  79.7

Dataset: Wine
dimension: 13
Total Acc:  63.888888

### Run the following cell for solving knapsack problem

In [7]:
file_path = 'Data/Knapsack/'
file_names = ["ks_8a", "ks_8b", "ks_8c", "ks_8d", "ks_8e", "ks_12a", "ks_12b", "ks_12c", "ks_12d", "ks_12e", "ks_16a", "ks_16b", "ks_16c", "ks_16d", "ks_16e", "ks_20a", "ks_20b", "ks_20c", "ks_20d", "ks_20e", "ks_24a", "ks_24b", "ks_24c", "ks_24d", "ks_24e"]

# set the global parameters
omega = 0.9                 
a2 = 1
a1 = 2
GP = 0.5
max_runs = 30
particle_count = 20
dimension = -1
max_iter = 500
pool_size = 30
fitness = fitness_Knapsack

weights = []
worth = []
max_weight = -1

for file_name in file_names:    
    fname = file_path + file_name + ".txt"     
    x = open(fname, 'r')
    count = int(x.readline())
    
    weights = np.array(list(map(int, x.readline().split())))
    worth = np.array(list(map(int, x.readline().split())))
    max_weight = int(x.readline())

    dimension = count
    
    population = initialize(particle_count, dimension)
    fitness_list = fitness(population)

    sorted_idx = np.argsort(fitness_list)
    population = population[sorted_idx, :].reshape((particle_count, dimension))
    fitness_list = fitness_list[0, sorted_idx]

    best_particle_BEO = np.zeros((max_runs, dimension))
    best_particle_BEOSA = np.zeros((max_runs, dimension))
    
    conv_BEO = np.zeros((max_runs, max_iter))
    conv_BEOSA = np.zeros((max_runs, max_iter))
    
    best_fitness_BEO = np.zeros((1, max_runs))
    best_fitness_BEOSA = np.zeros((1, max_runs))

    for run_no in range(max_runs):    
        best_particle_BEO[run_no,:], conv_BEO[run_no,:] = EO_SA(init_population=population, dimension=dimension, pool_size=pool_size, max_iter=max_iter, particle_count=particle_count, allow_SA=False)
        best_particle_BEOSA[run_no,:], conv_BEOSA[run_no,:] = EO_SA(init_population=population, dimension=dimension, pool_size=pool_size, max_iter=max_iter, particle_count=particle_count, allow_SA=True)
    
    best_fitness_BEO[0, :] = fitness(best_particle_BEO)
    best_fitness_BEOSA[0, :] = fitness(best_particle_BEOSA)
    
    print(-np.mean(best_fitness_BEO), np.std(best_fitness_BEO), -np.mean(best_fitness_BEOSA), np.std(best_fitness_BEOSA))
    
#     print("Dataset:%s  Mean:%f  Std:%f" %(file_name, -np.mean(best_fitness), np.std(best_fitness)))

FileNotFoundError: [Errno 2] No such file or directory: 'Data/Knapsack/ks_8a.txt'